In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import matplotlib.pyplot as plt

from train_dataset import *
from test_dataset import *

In [2]:
train_set = TrafficSignTrain()
test_set = TrafficSignTest()

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
#         a picture with resolution 64x64x3 fully connected to 43 classes
        self.fc = nn.Linear(64*64*3, 43)
        
    def forward(self, X):
#         flatten the input X
        X = X.view(-1, self.num_flat_features(X))
#         forward
        X = self.fc(X)
        return X
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
def train_data(checkpoint_path, model_name, learning_rate = (1e-4), num_epochs = 5, 
               batch_size = 1, resume = False, num_class = 43):
    net = Net()
    start = time.time()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    
    m = len(train_set)
    hist = []
    
    name_checkpoint = model_name + ".chkpt"
    if (resume):
        checkpoint = torch.load(os.path.join(checkpoint_path, name_checkpoint))
        hist = checkpoint["hist"]
        optimizer.load_state_dict(checkpoint["optimizer"])
        net.load_state_dict(checkpoint["net"])
    
    for iters in range(num_epochs):
        indices = np.random.permutation(m)
        for i in range(m):
            input = train_set[indices[i]-1]['image']
            target = train_set[indices[i]-1]['classId']
            
            input = Variable(torch.from_numpy(input).float())
            target = Variable(torch.from_numpy(target).long())
            
            out = net(input)
            loss = criterion(out, target)
            optimizer.zero_grad()
#             net.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i%4900==0):
                now = time.time()
                print("iterate %d: loss = %.9f, spent = %.5f" % (i + 1, loss.data[0], now - start))
                start = now
                
        print("Epoch %d: loss = %.9f" % (iters + 1, loss.data[0]))
        hist.append(loss.data[0])
    
    checkpoint = {
        "optimizer": optimizer.state_dict(),
        "hist": hist,
        "net": net.state_dict()
    }
    torch.save(checkpoint, os.path.join(checkpoint_path, name_checkpoint))
    
    return hist

In [ ]:
hist = train_data("../checkpoint", "NoneHiddenlayer", resume = False, num_epochs = 1)

iterate 1: loss = 3.766139507, spent = 0.01574
iterate 4901: loss = 0.950668514, spent = 55.13920
iterate 9801: loss = 1.850159049, spent = 77.60526
iterate 14701: loss = 1.904563069, spent = 47.63952
iterate 19601: loss = 5.319349766, spent = 45.87490
iterate 24501: loss = 0.019266386, spent = 43.65016
iterate 29401: loss = 0.011423065, spent = 53.14657
